In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install opencv-python mediapipe numpy pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 42.4 MB/s eta 0:00:00


In [ ]:
import os
import cv2
import numpy as np
import csv
import mediapipe as mp

In [ ]:
# MediaPipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.6)

In [ ]:
def compute_fourier_features_extended(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    f_transform = np.fft.fft2(gray)
    f_shift = np.fft.fftshift(f_transform)

    # Magnitude Spectrum
    magnitude_spectrum = np.abs(f_shift)
    log_magnitude = 20 * np.log(magnitude_spectrum + 1)

    # Thống kê cơ bản
    mean_val = np.mean(log_magnitude)
    max_val = np.max(log_magnitude)
    min_val = np.min(log_magnitude)
    var_val = np.var(log_magnitude)
    skew_val = np.mean((log_magnitude - mean_val)**3) / (np.var(log_magnitude)**1.5)
    kurtosis_val = np.mean((log_magnitude - mean_val)**4) / (np.var(log_magnitude)**2)
    energy_val = np.sum(log_magnitude ** 2)

    # Phân vùng phổ
    center = log_magnitude.shape[0] // 2
    low_region = log_magnitude[center - 10:center + 10, center - 10:center + 10]
    mid_region = log_magnitude[center - 30:center + 30, center - 30:center + 30]
    high_region = log_magnitude

    low_mean = np.mean(low_region)
    mid_mean = np.mean(mid_region)
    high_mean = np.mean(high_region)

    # Kết hợp tất cả các đặc trưng
    features = [
        mean_val, max_val, min_val, var_val, skew_val, kurtosis_val,
        energy_val, low_mean, mid_mean, high_mean
    ]
    return features

In [ ]:
# Hàm trích xuất đặc trưng khung xương từ MediaPipe
def compute_pose_features(image):
    rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = pose.process(rgb_image)
    if results.pose_landmarks:
        landmarks = results.pose_landmarks.landmark
        coords = [(lm.x, lm.y, lm.z) for lm in landmarks]
        return np.array(coords).flatten()  # Vector hóa tọa độ (x, y, z)
    else:
        return np.zeros(33 * 3)  # 33 khớp, mỗi khớp có (x, y, z)


In [ ]:
def compute_optical_flow_features(prev_frame, curr_frame):
    # Chuyển khung hình về dạng grayscale
    prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
    curr_gray = cv2.cvtColor(curr_frame, cv2.COLOR_BGR2GRAY)

    # Tính toán Optical Flow
    flow = cv2.calcOpticalFlowFarneback(prev_gray, curr_gray, None,
                                        pyr_scale=0.5, levels=3, winsize=15,
                                        iterations=3, poly_n=5, poly_sigma=1.2,
                                        flags=0)

    # Tách thành 2 thành phần: độ lớn (magnitude) và góc (angle)
    mag, ang = cv2.cartToPolar(flow[..., 0], flow[..., 1])

    # Trích xuất các đặc trưng thống kê từ độ lớn và góc
    mag_mean = np.mean(mag)
    mag_max = np.max(mag)
    mag_var = np.var(mag)

    ang_mean = np.mean(ang)
    ang_var = np.var(ang)

    return [mag_mean, mag_max, mag_var, ang_mean, ang_var]

In [ ]:
def apply_ema(prev_value, current_value, alpha):
    if prev_value is None:
        return current_value  # Nếu khung đầu tiên, gán giá trị hiện tại
    return alpha * current_value + (1 - alpha) * prev_value

In [ ]:
# Hàm xử lý một video
def process_video(name, video_path, label, csv_writer, step=6):
    cap = cv2.VideoCapture(video_path)
    frame_idx = 0
    prev_frame = None
    fourier_features = [] # Initialize as a list
    pose_features = [] # Initialize as a list
    optical_flow_features = [] # Initialize as a list

    ema_landmarks = None
    alpha = 0.2

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Chỉ xử lý các frame có chỉ số chia hết cho step
        if frame_idx % step == 0:

            frame = cv2.GaussianBlur(frame,(5,5),0)
            # Fourier features
            fourier_feat = compute_fourier_features_extended(frame)


            # Pose features
            pose_feat = compute_pose_features(frame)
                    # Áp dụng EMA để làm mịn
            ema_landmarks = apply_ema(ema_landmarks, pose_feat, alpha)

            # Optical Flow features
            optical_feat = np.zeros(5)
            if prev_frame is not None:
                optical_feat = compute_optical_flow_features(prev_frame, frame)

            prev_frame = frame


            row = [name]+ [frame_idx] + list(fourier_feat) + list(ema_landmarks) + list(optical_feat) + [label]

            csv_writer.writerow(row)

        frame_idx += 1



    cap.release()

In [ ]:
# Hàm xử lý toàn bộ dữ liệu
def process_dataset(dataset_path, output_csv, step=1):
    actions = os.listdir(dataset_path)

    # Định nghĩa cột tiêu đề
    column_names = (
        ['video_name']+
        ['Framecount']+
        [f'fourier_{i}' for i in range(10)] +  # 10 đặc trưng Fourier
        [f'pose_{i}' for i in range(33 * 3)] +  # 33 khớp (x, y, z)
        ['optical_mag_mean', 'optical_mag_max', 'optical_mag_var', 'optical_ang_mean', 'optical_ang_var'] +  # 5 đặc trưng Optical Flow
        ['label']
    )

    # Mở file CSV để ghi
    with open(output_csv, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(column_names)  # Ghi tiêu đề cột

        for label, action in enumerate(actions):
            action_folder = os.path.join(dataset_path, action)
            for video_file in os.listdir(action_folder):
                video_path = os.path.join(action_folder, video_file)
                name= action + "_" + video_file
                print(f"Processing {video_path}...")
                process_video(name,video_path, action, writer, step=step)
            print(f"finish {action}...")
    print(f"Features saved to {output_csv}")


In [ ]:
datavideo = "/content/drive/MyDrive/dltest/doantotnghiep/videoprocessing"
csvfile = "/content/drive/MyDrive/dltest/doantotnghiep/featureFourier_Pose_OF.csv"
process_dataset(datavideo, csvfile, step=5)
print("finish")

Processing /content/drive/MyDrive/dltest/doantotnghiep/videoprocessing/walk/training_84.avi...
Processing /content/drive/MyDrive/dltest/doantotnghiep/videoprocessing/walk/training_358.avi...
Processing /content/drive/MyDrive/dltest/doantotnghiep/videoprocessing/walk/training_73.avi...
Processing /content/drive/MyDrive/dltest/doantotnghiep/videoprocessing/walk/testing_69.avi...
Processing /content/drive/MyDrive/dltest/doantotnghiep/videoprocessing/walk/training_11.avi...
Processing /content/drive/MyDrive/dltest/doantotnghiep/videoprocessing/walk/training_1.avi...
Processing /content/drive/MyDrive/dltest/doantotnghiep/videoprocessing/walk/testing_74.avi...
Processing /content/drive/MyDrive/dltest/doantotnghiep/videoprocessing/walk/testing_91.avi...
Processing /content/drive/MyDrive/dltest/doantotnghiep/videoprocessing/walk/testing_59.avi...
Processing /content/drive/MyDrive/dltest/doantotnghiep/videoprocessing/walk/testing_54.avi...
Processing /content/drive/MyDrive/dltest/doantotnghiep/

In [ ]:
import pandas as pd

df = pd.read_csv("/content/drive/MyDrive/dltest/doantotnghiep/featureFourier_Pose_OF.csv")
df

,video_name,Framecount,fourier_0,fourier_1,fourier_2,fourier_3,fourier_4,fourier_5,fourier_6,fourier_7,...,pose_95,pose_96,pose_97,pose_98,optical_mag_mean,optical_mag_max,optical_mag_var,optical_ang_mean,optical_ang_var,label
0,walk_training_84.avi,0,106.185311,294.959586,7.365518,1095.189329,1.032723,3.854092,1.282574e+09,155.437301,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,walk
1,walk_training_84.avi,5,106.381614,294.950201,12.523801,1087.236481,1.042045,3.867230,1.286076e+09,155.427936,...,0.000000,0.000000,0.000000,0.000000,0.074855,2.888181,0.065008,3.088735,3.245488,walk
2,walk_training_84.avi,10,106.375573,294.983934,8.024693,1081.219407,1.052421,3.881832,1.285319e+09,155.350928,...,0.000000,0.000000,0.000000,0.000000,0.088426,3.459028,0.054950,3.127283,3.151580,walk
3,walk_training_84.avi,15,106.597392,294.958398,9.317518,1078.498106,1.035264,3.874825,1.289935e+09,154.555452,...,0.000000,0.000000,0.000000,0.000000,0.081930,3.464704,0.045617,3.162206,3.162373,walk
4,walk_training_84.avi,20,106.562877,295.004726,9.038766,1078.263082,1.048534,3.877546,1.289148e+09,155.083898,...,0.000000,0.000000,0.000000,0.000000,0.094871,3.883901,0.072297,2.948807,3.542318,walk
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11493,stand_training_51.avi,20,109.792374,326.043185,3.676385,973.331156,0.934019,3.760213,1.350712e+09,144.688776,...,-0.031349,0.277486,0.355259,-0.000558,0.586452,16.517130,4.491963,3.133084,3.676971,stand
11494,stand_training_51.avi,25,109.453236,326.012820,10.090451,985.208931,0.943161,3.779278,1.344234e+09,143.410337,...,-0.064483,0.343066,0.448150,-0.001325,0.673828,13.349601,2.768488,1.946067,3.155381,stand
11495,stand_training_51.avi,30,109.420751,325.998632,11.511304,976.284866,0.965913,3.844658,1.342572e+09,141.797394,...,-0.095289,0.391213,0.524219,-0.009435,0.634586,11.709802,1.856832,1.914812,2.155686,stand
11496,stand_training_51.avi,35,109.803615,325.988069,11.379275,978.772125,0.946930,3.791433,1.351532e+09,142.950534,...,-0.098948,0.431136,0.576822,-0.012536,0.530871,9.347586,1.965802,2.427384,2.541368,stand


In [ ]:
df.iloc[1:61]

,video_name,Framecount,fourier_0,fourier_1,fourier_2,fourier_3,fourier_4,fourier_5,fourier_6,fourier_7,...,pose_95,pose_96,pose_97,pose_98,optical_mag_mean,optical_mag_max,optical_mag_var,optical_ang_mean,optical_ang_var,label
1,walk_training_84.avi,5,106.381614,294.950201,1.252380e+01,1087.236481,1.042045,3.867230,1.286076e+09,155.427936,...,0.000000,0.000000,0.000000,0.000000,0.074855,2.888181,0.065008,3.088735,3.245488,walk
2,walk_training_84.avi,10,106.375573,294.983934,8.024693e+00,1081.219407,1.052421,3.881832,1.285319e+09,155.350928,...,0.000000,0.000000,0.000000,0.000000,0.088426,3.459028,0.054950,3.127283,3.151580,walk
3,walk_training_84.avi,15,106.597392,294.958398,9.317518e+00,1078.498106,1.035264,3.874825,1.289935e+09,154.555452,...,0.000000,0.000000,0.000000,0.000000,0.081930,3.464704,0.045617,3.162206,3.162373,walk
4,walk_training_84.avi,20,106.562877,295.004726,9.038766e+00,1078.263082,1.048534,3.877546,1.289148e+09,155.083898,...,0.000000,0.000000,0.000000,0.000000,0.094871,3.883901,0.072297,2.948807,3.542318,walk
5,walk_training_84.avi,25,106.420565,294.953879,5.755396e-12,1088.661203,1.030645,3.857991,1.287083e+09,155.198368,...,0.000000,0.000000,0.000000,0.000000,0.091653,2.191928,0.038079,3.194553,3.362944,walk
6,walk_training_84.avi,30,106.571799,294.927176,5.058562e+00,1083.759386,1.034357,3.859042,1.289915e+09,155.158882,...,0.000000,0.000000,0.000000,0.000000,0.056671,3.239808,0.040931,3.012042,3.727206,walk
7,walk_training_84.avi,35,106.564337,294.957516,5.539881e+00,1083.662505,1.037318,3.861094,1.289740e+09,155.361878,...,0.000000,0.000000,0.000000,0.000000,0.085580,3.313739,0.051760,3.074882,3.499551,walk
8,walk_training_84.avi,40,106.644554,294.923360,8.780293e+00,1080.632657,1.038614,3.858324,1.291199e+09,155.412252,...,0.000000,0.000000,0.000000,0.000000,0.097407,4.365277,0.073788,3.222754,3.864885,walk
9,walk_training_358.avi,0,111.682099,314.289241,1.386294e+01,1217.311934,0.870221,3.466266,9.988372e+08,184.436337,...,0.131712,0.317858,1.001434,0.126007,0.000000,0.000000,0.000000,0.000000,0.000000,walk
10,walk_training_358.avi,5,110.557922,314.737557,1.079215e+01,1220.215714,0.923926,3.581965,9.808210e+08,183.769681,...,0.149382,0.321114,1.002991,0.121695,2.474604,13.142626,1.427324,1.113611,3.642851,walk
